In [1]:
import pandas as pd
import numpy as np 
from rdkit.Chem import AllChem, PandasTools
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split


In [2]:
train_descriptors = pd.read_csv("./train_with_descriptors.csv")
test_descriptors = pd.read_csv("./test_with_descriptors.csv")

In [3]:
le = LabelEncoder()

In [4]:
# Replacing null values with mean based on techniques in titanic dataset 
train_dataset = train_descriptors.fillna(train_descriptors.mean())
test_dataset = test_descriptors.fillna(test_descriptors.mean())

In [5]:
# Drop the "Expected" column from the DataFrame
X = train_dataset.drop("Expected", axis=1)
y = train_dataset["Expected"]

In [6]:
y = le.fit_transform(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [8]:
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier

model =XGBClassifier(n_estimators=500, random_state=12)

rfe_method = RFE(
    model,
    n_features_to_select=50,
    step=2,
)

In [339]:
X_train_selected = rfe_method.transform(X_train)
X_test_selected = rfe_method.transform(X_test)
test_dataset = rfe_method.transform(test_dataset)

In [194]:
##from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

clf = XGBClassifier(n_estimators= 1700, max_depth = 6, random_state = 50,learning_rate = 0.01)
clf.fit(X_train_selected, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1700, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=50, ...)

In [195]:
predict = clf.predict(test_dataset)
predict2 = le.inverse_transform(predict)
predict2

array([2, 2, 2, ..., 2, 2, 2])

In [196]:
predict = clf.predict(X_test_selected)

In [197]:
from sklearn.metrics import f1_score

f1_score(y_test, predict, average= None).mean()

0.7303165948179433

In [295]:
print(np.count_nonzero(predict2==1))
print(np.count_nonzero(predict2==2))

882
10112


In [141]:
# Create submission file 
submission_file = pd.read_csv("./sample_submission.csv")

In [142]:
submission_file

,Id,Predicted
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,1
1,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,2
2,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,1
3,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+];1850,2
4,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,2
...,...,...
10989,CC(=NOCC1=CC=CC=C1C(=NOC)C(=O)OC)C2=CC(=CC=C2)...,1
10990,C1=CC=C(C(=C1)C(C2=CC=C(C=C2)O)C3=CC=C(C=C3)O)...,2
10991,CC1=NC(=NC(=N1)OC)NC(=O)NS(=O)(=O)C2=CC=CC=C2C...,1
10992,COP(=O)(OC)OC=C(Cl)Cl;28,2


In [143]:
submission_file["Predicted"] = predict2
submission_file.to_csv("rdkit_submission_42.csv",index=False)